In [2]:
import pyranges as pr
import pandas as pd
import seaborn as sns
import numpy as np
import sqlite3
import gffutils
import time
import pybedtools

In [3]:
start_time = time.time()
mouse_gr = pr.read_gtf("/home/U210050044/data/DH607-project/gencode.vM36.annotation.gtf")
end_time = time.time()
print(f"Finished reading into pyranges object in time {end_time-start_time}")

Finished reading into pyranges object in time 44.9543559551239


In [4]:
start_time = time.time()
mouse_db = gffutils.create_db("/home/U210050044/data/DH607-project/gencode.vM36.annotation.gtf","/home/U210050044/data/DH607-project/mouse_db.sqlite3",disable_infer_genes=True,disable_infer_transcripts=True, force=True)
end_time = time.time()
print(f"Finished converting gtf to sql db in time {end_time-start_time}")

Finished converting gtf to sql db in time 319.6335127353668


In [5]:
# Open the database
mouse_db = gffutils.FeatureDB("/home/U210050044/data/DH607-project/mouse_db.sqlite3")

In [6]:
conn = sqlite3.connect(":memory:")
start_time = time.time()
mouse_gr.df.to_sql("mouse", conn, if_exists="replace")
end_time = time.time()
print(f"Converted Pyranges object to in-memory sql db in time {end_time-start_time}")
cur = conn.cursor()

Converted Pyranges object to in-memory sql db in time 36.47839665412903


In [7]:
# Aggregate Queries: Counting transcripts, calculating exon lengths, and grouping features
### Aggregate Query 1: Count the number of exons for each gene
# Using pyranges
%timeit exon_counts_pr = (mouse_gr[mouse_gr.Feature == "exon"].df.groupby("gene_id").size().reset_index(name="exon_count"))

4.08 s ± 29.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# Using gffutils
def exon_counts_gffutls():
    # Iterate over all genes and count their exons
    exon_counts = {}
    for gene in mouse_db.features_of_type("gene"):
        # Efficiently count exons for each gene using the `children` method
        exon_counts[gene.id] = sum(
            1 for _ in mouse_db.children(gene, featuretype="exon")
    )
    return exon_counts
%timeit exon_counts_gff = exon_counts_gffutls()

34.2 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
# Using sql
%timeit exon_counts_sql = pd.read_sql_query("SELECT gene_id, COUNT(*) as exon_count FROM mouse WHERE Feature = 'exon' GROUP BY gene_id", conn)

1.4 s ± 137 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
### Aggregate Query 2: Calculate the total length of exons for each gene
# Using pyranges
%timeit exon_lengths_pr = (mouse_gr[mouse_gr.Feature == "exon"].df.assign(length=lambda df: df["End"] - df["Start"]).groupby("gene_id")["length"].sum().reset_index(name="total_exon_length"))

4.81 s ± 196 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# Using gffutils
def exon_lengths_gffutls():
    # Dictionary to store total exon length per gene
    exon_lengths = {}
    # Iterate over all genes and calculate total exon length
    for gene in mouse_db.features_of_type("gene"):
        # Compute total exon length for each gene
        exon_lengths[gene.id] = sum(
            (child.end - child.start + 1)
            for child in mouse_db.children(gene, featuretype="exon")
        )
    return exon_lengths
%timeit exon_lengths_gff = exon_lengths_gffutls()

34.7 s ± 52.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# using sql
%timeit exon_lengths_sql = pd.read_sql_query("SELECT gene_id, SUM(End - Start) as total_exon_length FROM mouse WHERE Feature = 'exon' GROUP BY gene_id", conn)

1.51 s ± 8.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
### Aggregate Query 3: Identify the chromosome with the highest number of transcripts
# Using pyranges
%timeit transcript_counts_pr = mouse_gr[mouse_gr.Feature == "transcript"].df["Chromosome"].value_counts().idxmax()

915 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
# Using gffutils
def transcript_counts_gffutls():
    # Dictionary to store the number of transcripts per chromosome
    transcript_counts = {}
    # Iterate over all transcripts and count the number of transcripts per chromosome
    for transcript in mouse_db.features_of_type("transcript"):
        chrom = transcript.chrom
        if chrom not in transcript_counts:
            transcript_counts[chrom] = 0
        transcript_counts[chrom] += 1
    return max(transcript_counts, key=transcript_counts.get)
%timeit transcript_counts_gff = transcript_counts_gffutls()

5.77 s ± 26.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
# using sql
%timeit transcript_counts_sql = pd.read_sql_query("SELECT Chromosome, COUNT(*) as transcript_count FROM mouse WHERE Feature = 'transcript' GROUP BY Chromosome ORDER BY transcript_count DESC LIMIT 1", conn)

542 ms ± 8.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Interval Arithmetic

In [16]:
# Query 1: Merging Overlapping Exon Intervals
# Combine all overlapping exon intervals into contiguous regions
# Using pyranges
%timeit exon_intervals_pr = mouse_gr[mouse_gr.Feature == "exon"].merge(strand=False)

3.23 s ± 15.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
# using gffutils TODO: run
def exon_intervals_gffutls():
    exon_intervals = [
        gffutils.helpers.asinterval(exon) for exon in mouse_db.features_of_type("exon")
    ]
    exon_intervals_sorted = sorted(exon_intervals, key=lambda x: (x.chrom, x.start))
    # Use pybedtools to merge overlapping intervals
    return pybedtools.BedTool(exon_intervals_sorted).merge()
%timeit exon_intervals_gff = exon_intervals_gffutls()

1min 5s ± 776 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# using sql
def find_exon_intervals_sql():
    exon_intervals = pd.read_sql_query("SELECT Chromosome, Start, End FROM mouse WHERE Feature = 'exon' ORDER BY Chromosome, Start", conn)
    return pybedtools.BedTool.from_dataframe(exon_intervals).merge()
%timeit exon_intervals_sql = find_exon_intervals_sql()

6.46 s ± 34.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
# Query 2: Finding Overlaps with a Specific Interval
# Find all gene features that overlap a given interval chr1:100000-200000
# Using pyranges
%timeit overlapping_genes_pr = mouse_gr[mouse_gr.Feature == "gene"].merge(strand=False).overlap(pr.from_dict({"Chromosome": ["1"], "Start": [100000], "End": [200000]}))

519 ms ± 2.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
# using gffutils TODO: run
def overlapping_genes_gffutls():
    exon_intervals_overlapping = pybedtools.BedTool.from_dataframe(pd.DataFrame({"Chromosome": ["1"], "Start": [100000], "End": [200000]}))
    exon_intervals = [
        gffutils.helpers.asinterval(exon) for exon in mouse_db.features_of_type("exon")
    ]
    exon_intervals_sorted = sorted(exon_intervals, key=lambda x: (x.chrom, x.start))
    # Use pybedtools to merge overlapping intervals
    return exon_intervals_overlapping.window(pybedtools.BedTool(exon_intervals_sorted).merge(), w=0)
%timeit overlapping_genes_gff = overlapping_genes_gffutls()

1min 4s ± 157 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
# using sql
def find_overlapping_genes_sql():
    exon_intervals_overlapping = pybedtools.BedTool.from_dataframe(pd.DataFrame({"Chromosome": ["1"], "Start": [100000], "End": [200000]}))
    exon_intervals = pd.read_sql_query("SELECT Chromosome, Start, End FROM mouse WHERE Feature = 'exon' ORDER BY Chromosome, Start", conn)
    return exon_intervals_overlapping.window(pybedtools.BedTool.from_dataframe(exon_intervals).merge(), w=0)
%timeit overlapping_genes_sql = find_overlapping_genes_sql()

6.73 s ± 36.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
# Query 3: Subtracting Intervals
# Remove a set of repetitive regions from the exon features.
# Using pyranges
%timeit subtracted_intervals_pr = mouse_gr[mouse_gr.Feature == "exon"].merge(strand=False).subtract(pr.from_dict({"Chromosome": ["1", "1"],"Start": [12000000, 16000000],"End": [12800000, 16800000],}))

3.43 s ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
# using gffutils TODO: run
def subtracted_intervals_gffutls():
    exon_intervals_subtract = pybedtools.BedTool.from_dataframe(pd.DataFrame({"Chromosome": ["1", "1"], "Start": [12000000, 16000000],"End": [12800000, 16800000],}))
    exon_intervals = [
        gffutils.helpers.asinterval(exon) for exon in mouse_db.features_of_type("exon")
    ]
    exon_intervals_sorted = sorted(exon_intervals, key=lambda x: (x.chrom, x.start))
    # Use pybedtools to merge overlapping intervals
    return pybedtools.BedTool(exon_intervals_sorted).merge().subtract(exon_intervals_subtract)
%timeit overlapping_genes_gff = overlapping_genes_gffutls()

1min 4s ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
# using sql
def find_subtracted_intervals_sql():
    exon_intervals_subtract = pybedtools.BedTool.from_dataframe(pd.DataFrame({"Chromosome": ["1", "1"], "Start": [12000000, 16000000],"End": [12800000, 16800000],}))
    exon_intervals = pd.read_sql_query("SELECT Chromosome, Start, End FROM mouse WHERE Feature = 'exon' ORDER BY Chromosome, Start", conn)
    # Use pybedtools to merge overlapping intervals
    return pybedtools.BedTool.from_dataframe(exon_intervals).merge().subtract(exon_intervals_subtract)
%timeit subtracted_intervals_sql = find_subtracted_intervals_sql()

***** WARNING: File /tmp/pybedtools.4qvxs1b6.tmp has inconsistent naming convention for record:
chr1	3143475	3144545

***** WARNING: File /tmp/pybedtools.4qvxs1b6.tmp has inconsistent naming convention for record:
chr1	3143475	3144545

***** WARNING: File /tmp/pybedtools.882e1vdu.tmp has inconsistent naming convention for record:
chr1	3143475	3144545

***** WARNING: File /tmp/pybedtools.882e1vdu.tmp has inconsistent naming convention for record:
chr1	3143475	3144545

***** WARNING: File /tmp/pybedtools.f6to89k4.tmp has inconsistent naming convention for record:
chr1	3143475	3144545

***** WARNING: File /tmp/pybedtools.f6to89k4.tmp has inconsistent naming convention for record:
chr1	3143475	3144545

***** WARNING: File /tmp/pybedtools._nvwvyls.tmp has inconsistent naming convention for record:
chr1	3143475	3144545

***** WARNING: File /tmp/pybedtools._nvwvyls.tmp has inconsistent naming convention for record:
chr1	3143475	3144545

***** WARNING: File /tmp/pybedtools.sehw92yq.tmp has inc

6.83 s ± 40.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


***** WARNING: File /tmp/pybedtools.iuxdt0if.tmp has inconsistent naming convention for record:
chr1	3143475	3144545

***** WARNING: File /tmp/pybedtools.iuxdt0if.tmp has inconsistent naming convention for record:
chr1	3143475	3144545

